# Game Builder

## Setup

### Install Dependencies

In [ ]:
%pip install nltk
%pip install langchain
%pip install openai
#%pip install marisa-trie
#%pip install graphviz
#%pip install transformers
#%pip install scikit-learn


# # For calculating cosine similarity
# %pip install chromadb
# %pip install tiktoken

# For safe eval
%pip install simpleeval

### Import libraries

In [ ]:
%load_ext autoreload
%autoreload 2
from langchain.schema.language_model import BaseLanguageModel
# from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
# from langchain.prompts import FewShotChatMessagePromptTemplate, ChatPromptTemplate
# from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.prompt import PromptTemplate
from langchain import LLMChain, PromptTemplate
from typing import *
import sys, json, os, random, re, math
from copy import deepcopy
from utils import *
import nltk # type: ignore
nltk.download('wordnet') # type: ignore


### Load LLM (Choose 1 only)

#### GPT-3.5 or 4o-mini

In [ ]:
from llm.chatgpt import ChatGPT
llm = ChatGPT("gpt-4o-mini", temperature = 0)

#### Llama-2

In [ ]:
# GPU llama-cpp-python
#!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python==0.1.78 --force-reinstall --upgrade --no-cache-dir --verbose
# For downloading the model
#!pip install huggingface_hub
# The interface we use to interact with the LLM
#!pip -q install langchain

### Generate Game

#### We use the output from the neural planner as the input.

In [4]:
game_input = {
    "game_logics": {
        "adventurer speak with the village elders.": {
            "item needed": [],
            "location": [
                "at village hall"
            ],
            "preceding_events": [],
            "description": [],
            "results": []
        },
        "adventurer read books.": {
            "item needed": [],
            "location": [
                "at library"
            ],
            "preceding_events": [],
            "description": [
                "There is a book at the library."
            ],
            "results": [
                "has book"
            ]
        },
        "adventurer find maps.": {
            "item needed": [
                "has books"
            ],
            "location": [
                "at library"
            ],
            "preceding_events": [],
            "description": [
                "There is a map at the library."
            ],
            "results": [
                "has map"
            ]
        },
        "adventurer take torch.": {
            "item needed": [
                "has money"
            ],
            "location": [
                "at general store"
            ],
            "preceding_events": [],
            "description": [
                "There is a torch for sale at the general"
            ],
            "results": [
                "has torch"
            ]
        },
        "adventurer take basket.": {
            "item needed": [],
            "location": [
                "at forest"
            ],
            "preceding_events": [],
            "description": [
                "There is a basket in the forest."
            ],
            "results": [
                "has basket"
            ]
        },
        "adventurer gather ingredients for a cure.": {
            "item needed": [
                "has basket"
            ],
            "location": [
                "at forest"
            ],
            "preceding_events": [],
            "description": [],
            "results": []
        },
        "adventure meet with a wise woman in the forest.": {
            "item needed": [],
            "location": [
                "at forest"
            ],
            "preceding_events": [
                "adventurer gather ingredients for a cure."
            ],
            "description": [],
            "results": []
        }
    },
    "map": {
        "at village hall": [
            {
                "type": "npc",
                "content": "has village elders"
            }
        ],
        "at forest": [
            {
                "type": "npc",
                "content": "has farmers"
            },
            {
                "type": "npc",
                "content": "has herbalist"
            },
            {
                "type": "npc",
                "content": "has witch"
            },
            {
                "type": "npc",
                "content": "has wise woman"
            },
            {
                "type": "item",
                "content": "has basket"
            },
        ],
        "at general store": [
            {
                "type": "item",
                "content": "has torch"
            },
            {
                "type": "item",
                "content": "has lockpicking tool"
            },
            {
                "type": "npc",
                "content": "has merchant"
            }
        ],
        "at library": [
            {
                "type": "item",
                "content": "has maps"
            },
            {
                "type": "item",
                "content": "has books"
            }
        ],
        "at village": [
            {
                "type": "npc",
                "content": "has villagers"
            }
        ]
    }
}

In [ ]:
with open("result/The_Curse_of_Werewolf.json", "r") as f:
    game_input = json.load(f)
print(game_input)

#### Some necessary functions for us to generate a playable game.

In [ ]:
from game_construct_prompt import expand_sentence, analyze_action, populate_attribute, get_verbs, generate_new_preconditions, check_if_existing_action, check_future_events
from game import Game
from world import World
from nodes import Node, Item, Room, Character, Player, ContainerItem
from logic_template import ActionTemplate, EventTemplate
from condition import ComplexCondition
from type import Coordinate
import numpy as np
import traceback


class GameBuilderHelper:

    @staticmethod
    def get_topological_ordering_of_game_logics(game_input: Dict[str, Any]) -> Dict[str, Any]:
        """
        Returns a topological ordering of the game logics. Events that must happen before other events are ordered first.

        Args:
            game_input (Dict[str, Any]): Dictionary containing game's logic definitions.

        Returns:
            Dict[str, Any]: Dictionary containing game's logic definitions with events ordered topologically.
        """
        game_logics = game_input['game_logics']

        # Define the possible states for nodes during DFS
        UNVISITED = 0
        VISITING = 1
        VISITED = 2

        # Initialization
        order: List[str] = []  # Stores the topological order of nodes (events)
        # Track state of each node
        state = {event: UNVISITED for event in game_logics}

        def dfs(event: str) -> bool:
            """Recursive helper function to perform DFS and check for cycles."""
            if state[event] == VISITING:
                # Cycle detected, topological order is not possible
                return False
            if state[event] == VISITED:
                # Already processed, skip
                return True

            # Mark current node as VISITING
            state[event] = VISITING

            # Visit all neighbors
            for preceding_event in game_logics[event]['preceding_events']:
                if not dfs(preceding_event):
                    return False

            # Mark current node as VISITED and add to order
            state[event] = VISITED
            order.append(event)

            return True

        # For each node, if it's unvisited, perform DFS
        for event in game_logics:
            if state[event] == UNVISITED and not dfs(event):
                # If there's a cycle, return an empty list or raise an error
                raise ValueError(
                    "Cycle detected, topological order is not possible.")
        
        # Return the game logics with events ordered topologically
        return {event: game_input['game_logics'][event] for event in order}

    @staticmethod
    def _expand_sentences(game_input: Dict[str, Any]) -> Tuple[Dict[str, str], Dict[str, List[str]]]:
        """
        Expands game logic sentences and extracts objects involved in them.

        For each sentence in game_input['game_logics'], it expands the sentence and identifies the objects 
        associated with the sentence. For example, "Adventurer crafts sword" will be expanded to "Adventurer crafts sword with iron and anvil."

        Args:
            game_input (Dict[str, Any]): Dictionary containing game's logic definitions.

        Returns:
            Tuple[Dict[str, str], Dict[str, List[str]]]: 
            - Dictionary mapping original sentences to their expanded form.
            - Dictionary mapping original sentences to objects involved in them.

        """
        expanded_sentences: Dict[str, str] = {}
        objects_in_expanded_sentences: Dict[str, List[str]] = {}
        for sentence in game_input['game_logics']:
            expanded_sentence, objects = (expand_sentence(
                llm, sentence, game_input['game_logics'][sentence]['item needed']))
            expanded_sentences[sentence] = expanded_sentence
            objects_in_expanded_sentences[sentence] = objects

        return expanded_sentences, objects_in_expanded_sentences

    @staticmethod
    def expand_game_logics(game_input: Dict[str, Any]) -> Dict[str, Any]:
        """
        Updates game logic with expanded sentences.

        For each game logic sentence, the function replaces original sentences with their expanded form 
        and updates any preceding events.

        Args:
            game_input (Dict[str, Any]): Original game logic definitions.
            expanded_sentences (Dict[str, str]): Dictionary mapping original sentences to their expanded form.

        Returns:
            Dict[str, Any]: Updated game logics.
        """
        expanded_sentences, _ = GameBuilderHelper._expand_sentences(game_input)
        new_game_logics = deepcopy({expanded_sentence: game_input['game_logics'][original_sentence]
                                   for original_sentence, expanded_sentence in expanded_sentences.items()})
        for sentence in new_game_logics:
            old_preceding_events: List[str] = new_game_logics[sentence]['preceding_events']
            new_preceding_events: List[str] = []
            for old_preceding_event in old_preceding_events:
                
                if old_preceding_event in expanded_sentences:
                    new_preceding_events.append(
                        expanded_sentences[old_preceding_event])
                else:
                    print_warning(
                        f'Warning: {old_preceding_event} not in expanded_sentences!', 'update_game_with_expanded_sentences')
                    new_preceding_events.append(old_preceding_event)
            new_game_logics[sentence]['preceding_events'] = new_preceding_events
        return new_game_logics
    
    @staticmethod
    def add_nodes_from_game_logics(world: World, sentence: str, game_logics_content: Dict[str, Any], parsed_arguments_in_sentence: Dict[str, Union[str, List[str]]]) -> Room:

        # 1: Get the room where the action takes place. 
        # If it doesn't exist, add it to the world (this should not happen often).
        location_name = remove_extra_spaces(
            game_logics_content['location'][0])
        room = GameBuilderHelper.add_node_if_not_exist(world, location_name, Room,
                                                       (0, world.get_num_rooms()),
                                                       not_exist_callback=lambda: print_warning(
                                                           f'Warning: {location_name} appears in game logics but not in map! Adding {location_name} to world', 'get_world'))
        assert isinstance(room, Room)
        # 2: Get the items in "item needed".
        # If they don't exist, add them to the room.

        for item_name in game_logics_content['item needed']:
            GameBuilderHelper.add_node_if_not_exist(
                world, item_name, Item, room,
                not_exist_callback=lambda: print_warning(
                    f'Warning: {item_name} appears in game logics but not in map! Adding {item_name} to world', 'get_world')
            )           
        # 3: Get the characters and items involved in the sentence. 
        # If they don't exist, add them to the room.

        for placeholder in parsed_arguments_in_sentence:
            node_names = parsed_arguments_in_sentence[placeholder]
            nodes: List[Node] = []

            if isinstance(node_names, str):
                node_names = [node_names]
            if 'item' in placeholder:
                for node_name in node_names:
                    node = GameBuilderHelper.add_node_if_not_exist(
                        world, node_name, Item, room,
                        not_exist_callback=lambda: print_warning(
                            f'Warning: {node_name} appears in game logics but not in map! Adding {node_name} to world', 'get_world')
                    )
                    nodes.append(node)
            elif 'container' in placeholder:
                for node_name in node_names:
                    node = GameBuilderHelper.add_node_if_not_exist(
                        world, node_name, ContainerItem, room,
                        not_exist_callback=lambda: print_warning(
                            f'Warning: {node_name} appears in game logics but not in map! Adding {node_name} to world', 'get_world')
                    )
                    nodes.append(node)
            elif 'character' in placeholder:
                for node_name in node_names:
                    node = GameBuilderHelper.add_node_if_not_exist(
                        world, node_name, Character, room,
                        not_exist_callback=lambda: print_warning(
                            f'Warning: {node_name} appears in game logics but not in map! Adding {node_name} to world', 'get_world')
                    )
                    nodes.append(node)
            elif 'room' in placeholder:
                for room_name in node_names:
                    if not world.node_exists(room_name):
                        print_warning(
                            f'Warning: {room_name} is a room that does not appear in the map! Ignoring', 'get_world')
                    else:
                        nodes.append(world.find_node(room_name))
            else:
                print_warning(
                    f'Warning: {parsed_arguments_in_sentence[placeholder]} is not an item or character! Ignoring', 'get_world')
            
            # TODO: Populate the game with hidden items and characters that are not explicitly mentioned in the game logics or map.
            # This makes generation too slow, so removed from this version.
        return room

    @staticmethod
    def add_node_if_not_exist(world: World, name: str, type: Type[Node], container: Union[Coordinate, Node], description: str = '', exist_callback: Union[None, Callable[[], None]] = None, not_exist_callback: Union[None, Callable[[], None]] = None, node_added_callback: Union[None, Callable[[], None]] = None, **kwargs: Any) -> Node:
        name = remove_extra_spaces(name)
        if type == Room and name.lower().startswith('at '):
            name = name[3:]
        elif type in [Item, Character, ContainerItem] and name.lower().startswith('has '):
            name = name[4:]
        #name = get_lemma(name)
        if not description:
            # TODO: Better description
            if type == Room:
                description = 'You are at ' + name + '.'
            else:
                description = 'An ' + \
                    name if name[0].lower() in ['a', 'e', 'i', 'o',
                                                'u'] else 'A ' + name + '.'
        try:
            if type == Room:
                node = world.find_node(name, strict=True)
            else:
                assert isinstance(container, Node)
                try:
                    node = world.find_node(
                        name, room=container.get_room().id, local=True, strict=True)
                except:
                    node = world.find_node(name, strict=True)
            # node exists.
            if exist_callback:
                exist_callback()
            return node
        except:
            pass
        if not_exist_callback:
            not_exist_callback()
        node = type(name, description=description)
        for attribute_name, attribute_value in kwargs.items():
            node.set_attribute(attribute_name, attribute_value)
        world.add_node(node, container)
        if node_added_callback:
            node_added_callback()
        return node

class GameBuilder:
    def __init__(self, game_input: Dict[str, Any]) -> None:
        self.game_input = game_input
        world = World(configurations={"map_size": (1, 128)})
        self.game = Game(world, [], [])
        self.replay_games: List[Game] = [deepcopy(self.game)]

    def preprocess_game_input(self) -> None:
        game_input = self.game_input
        game_logics = game_input['game_logics']

        # Step 1: Get a topological ordering of game logics: Events that must happen before other events are ordered first.
        game_logics = GameBuilderHelper.get_topological_ordering_of_game_logics(
            game_input)

        # Step 2: Expand game logics: Add objects to sentences and expand them
        game_logics = GameBuilderHelper.expand_game_logics(game_input)

        game_input['game_logics'] = game_logics

        self.game_input = game_input

    def initialize_world(self) -> None:
        """
        Generates a game world based on game input.

        Only rooms, characters, and items defined in game_input['map'] are added to the world.

        Args:
            game_input (Dict[str, Any]): Dictionary containing game's configurations and logic definitions.

        Returns:
            World: The populated game world.

        """
        game_input = self.game_input

        world = self.game.world

        # Add rooms, characters, and items defined in game_input['map']
        for room_name in game_input['map']:
            pattern = r"\b(?:at|has)\b\s*"
            cleaned_room_name = re.sub(pattern, '', room_name).strip()
            room = GameBuilderHelper.add_node_if_not_exist(world, cleaned_room_name, Room, (
                0, world.get_num_rooms()), node_added_callback=lambda: log('get_world', f'Adding Room {cleaned_room_name} to world'))
            for node in game_input['map'][room_name]:
                node_name = node['content']
                cleaned_node_name = re.sub(pattern, '', node_name).strip()
                if node['type'] == 'npc':
                    GameBuilderHelper.add_node_if_not_exist(world, cleaned_node_name, Character, room, node_added_callback=lambda: log(
                        'get_world', f'Adding Character {cleaned_node_name} to world'))
                elif node['type'] == 'item':
                    GameBuilderHelper.add_node_if_not_exist(world, cleaned_node_name, Item, room, node_added_callback=lambda: log(
                        'get_world', f'Adding Item {cleaned_node_name} to world'))

        # TODO: Player goal and description should be provided or generated
        first_room = world.get_room_from_coordinate((0, 0))
        assert first_room is not None
        player = GameBuilderHelper.add_node_if_not_exist(world, 'player', Player, first_room, node_added_callback=lambda: log(
            'get_world', f'Adding Player to world'), goal="Your goal is resolve the curse of the werewolf")
        # TODO: This is a hack to make sure there is money in the game. Should be removed.
        GameBuilderHelper.add_node_if_not_exist(world, 'money', Item, player)

    def _check_action_effects(self, sentence: str) -> Tuple[bool, str]:
        """
        Checks if an action has the correct effects.

        The function checks if the action has the correct effects. If it does, it returns True. Otherwise, it returns False and a message explaining why the action is invalid.

        Args:
            sentence (str): The action to be checked.

        Returns:
            Tuple[bool, str]: 
            - True if the action has the correct effects, False otherwise.
            - A message explaining why the action is invalid if it's not added to the game.
        """
        raise NotImplementedError('Not implemented yet')
    
    def add_dynamic_action(self, user_input: str) -> Tuple[bool, str, str]:
        items, attributes, complete_sentences, output_list = generate_new_preconditions(self.game, llm, user_input, [])
        valid_rooms = []
        for node in adventureGame.world.nodes:
            if type(node) is Room:
                valid_rooms.append(node)
        for item in items:
            rand_room = np.random.choice(valid_rooms)
            GameBuilderHelper.add_node_if_not_exist(adventureGame.world, item, Item, rand_room, node_added_callback=lambda: log(
                        'get_world', f'Adding Item {item} to world'))

    # NOT TESTED!!!!!!!
    def _try_adding_action(self, sentence: str, previous_attempts:List[Tuple[str, str]]=[]) -> Tuple[bool, str, str, ActionTemplate]:
        """
        Tries to add an action to the game.

        The function tries to add an action to the game. If the action is valid, it adds it to the game and returns True.
        Otherwise, it returns False and a message explaining why the action is invalid.

        Args:
            sentence (str): The action to be added to the game.

        Returns:
            Tuple[bool, str, str]: 
            - True if the action is valid and added to the game, False otherwise.
            - LLM Raw Output
            - A message explaining why the action is invalid if it's not added to the game.
        """
        assert sentence in self.game_input['game_logics'], f'{sentence} not in game_input["game_logics"]'
        llm_raw_output = f'Error! Failed to analyze action: {sentence}'
        preceding_events = self.game_input['game_logics'][sentence]['preceding_events']
        input = sentence + "; room: " + self.game_input['game_logics'][sentence]['location'][0] + "."
        try:     
            llm_raw_output, base_form, arguments, attribute_list, action_template = analyze_action(self.game,
                llm, input, previous_attempts)  # Example: {"character": ["adventurer"], "item": ["money"]}
            log('_try_adding_action', f'\n{llm_raw_output}')
            world = self.game.world

            # STEP 1: Register attributes.
            for attribute_name, belonging_class_name, attribute_type in attribute_list:
                belonging_class = Node.mapping[belonging_class_name]
                if not belonging_class.has_attribute(attribute_name):
                    belonging_class.register_new_attribute(
                        attribute_name, attribute_type, None)

            # STEP 2: Identify nodes involved in the action. 
            # Add mising characters and items in game_input['game_logics']. 
            # This should not happen often.
            
            room = GameBuilderHelper.add_nodes_from_game_logics(world, sentence, self.game_input['game_logics'][sentence], arguments)

            # STEP 3: Move player to the room where the action takes place, assuming that the player can go there.
            # TODO: If the player can't go to the room, the action is invalid.
            player = world.player
            assert player is not None, 'Player is not in the world!'
            if player.get_room() != room:
                world.move_node(player, room)

            # STEP 4: Add action, and make sure the preconditions are met
            # Populate node attributes if they are not specified. objects in the sentence are assumed to satisfy the conditions in game logics.
            # In condition, if attribute is not specified (having a value of None), make it satisfy the condition.
            # check if the condition is a simple condition consisting of only the "and" operator.
            # If it is simple, edit node attributes to satisfy the condition.
            # If it is complex, prompt llm to edit node attributes to satisfy the condition.
            # If it still fails, return failure.

            self.game.add_action_template(action_template)
            command = replace_placeholders(base_form, arguments)
            action = self.game.get_action(command)
            action.input_name = sentence
            for flag in self.game_input['game_logics'][sentence]['preceding_events']:
                action.flags.append(flag)
            assert action is not None, "Bug! Action is none!"
            fixes = action.conditions.get_fixes_complex(self.game, llm=llm)
            for fix in fixes:
                fix.apply(self.game.world)
            
            action_result = action.execute(self.game)
            self.game.happened_events.add(sentence)

            # STEP 6: Check if the action succeeds.
            action_result = self.game.action_history[-1]
            if not action_result.success:
                # REMOVE THE ACTION IF ALREADY ADDED
                return False, llm_raw_output, action_result.observation, None

            # STEP 7: Check if the action has the correct effects.
            desired_effects: List[str] = self.game_input['game_logics'][sentence]['results']
            desired_effects = [f'{{{effect}}}' for effect in desired_effects]
            actual_effects_expression = ' and '.join(desired_effects)
            actual_effects_condition = ComplexCondition.build_from_string(
                game = self.game, expression = actual_effects_expression)
            is_satisfied, messages = actual_effects_condition.evaluate(
                self.game)

            #if not is_satisfied:
            #    return False, llm_raw_output, '\n'.join(messages)
            
        except Exception as e:
            log('_try_adding_action', f"Caught exception: {e}")
            traceback.print_exc()
            print("REMOVE NOT WORKING ACTION", self.game.remove_action(action_template))
            return False, llm_raw_output, str(e), None

        # Add command to list of ideal action
        self.game.commands.append(command)
        return True, llm_raw_output, '', action_template

    def save_game_snapshot(self) -> None:
        self.replay_games.append(deepcopy(self.game))

    def populate_node_attributes(self) -> None:
        nodes = self.game.world.nodes
        for node in nodes:
            _, attribute_values = populate_attribute(llm, node)
            for attribute, value in attribute_values.items():
                node.set_attribute(attribute, value)

    # NOT TESTED!!!
    def build_game(self) -> Game:


        # Step 1: Preprocess game input
        self.preprocess_game_input()

        # Step 2: Initialize world
        self.initialize_world()
        self.save_game_snapshot()

        # Step 3: Add actions
        look = ActionTemplate('look', 'Display {player.observation}', 'None')
        inventory = ActionTemplate('inventory', 'Display {inventory}')
        go = ActionTemplate('go to {room1}', 'Move {player} to {room1}; Display You have now entered {room1}; Display {player.observation}')
        #take = ActionTemplate('take {item1}', 'Move {item1} to {inventory}; Display You have now taken {item1}; Display {player.observation}')
        self.game.add_action_template(look)
        self.game.add_action_template(inventory)
        self.game.add_action_template(go)
        #self.game.add_action_template(take)

        MAXIMUM_ATTEMPT = 2
        success_count = 0.0
        count = 0.0
        length = 0.0
        failed_actions = []
        for sentence in self.game_input['game_logics']:
            length += 1
            previous_attempts: List[Tuple[str, str]] = []
            check = False
            for i in range(MAXIMUM_ATTEMPT):
                count += 1
                log('build_game', f'Adding sentence: {sentence}\tAttempt: {i+1}/{MAXIMUM_ATTEMPT}')
                is_successful, llm_raw_output, message, new_action_template = self._try_adding_action(sentence, previous_attempts=previous_attempts)
                check = check or is_successful
                if is_successful:
                    success_count += 1
                    break
                else:
                    previous_attempts.append((llm_raw_output, message))
            if not check:
                failed_actions.append(sentence)
            #    raise RuntimeError(f'Maximum number of attempts failed when adding action {sentence}!')
            self.save_game_snapshot()

        # Step 4: Restore game map to original state, and populate missing attributes

        room_count = 0
        item_count = 0
        char_count = 0

        for node in self.game.world.nodes:
            if type(node) is Room:
                room_count += 1
                continue
            if type(node) is Item:
                item_count += 1
                continue
            if type(node) is Character:
                char_count += 1
                continue
            #print(node, node.container, node.original_container)
        
        print("NODE COUNTS")
        print(room_count, char_count, item_count)

        self.game.world.restore_initial_state()
        self.game.happened_events = set()
        self.game.happened_actions = set()
        self.populate_node_attributes()
        
        return self.game


game_builder = GameBuilder(game_input)
adventureGame = game_builder.build_game()
game_builder.game.world.save('game_builder_output.json')
result = game_builder.game.world.serialize()
print(result)

In [6]:
def addNewAction(adventureGame, llm, user_input, currRoom):
    llm_raw_output, base_form, arguments, attribute_list, add_attribute_list, subject, preceding_events, effects, action_template = generate_new_preconditions(adventureGame, llm, user_input + "; room: " + currRoom, [])
    try:
        found = adventureGame.world.find_node(subject)
    except:
        print_warning("The subject of your action does not exist.")
        return
    valid_rooms = []
    for node in adventureGame.world.nodes:
        if type(node) is Room:
            valid_rooms.append(node)
    ###Adding new items randomly to the world
    for obj in arguments.values():
        try:
            found = adventureGame.world.find_node(obj)
        except:
            rand_room = np.random.choice(valid_rooms)
            new_obj = Item(obj, obj)
            adventureGame.world.add_node(new_obj, rand_room)
                        
    ###Registering new attributes with corresponding objects

    for attribute_name, belonging_class_name, attribute_type in attribute_list:
        belonging_class = Node.mapping[belonging_class_name]
        if not belonging_class.has_attribute(attribute_name):
            belonging_class.register_new_attribute(
                attribute_name, attribute_type, None)
    
    for add_attribute, belonging_class_name, attribute_type in add_attribute_list:
        attribute_input = subject + "." + add_attribute
        for action in adventureGame.actions.keys():
            if subject in action:
                is_necessary, new_attribute = check_future_events(llm, adventureGame.actions[action], attribute_input)
                proc = adventureGame.actions[action].conditions.processed_condition_expression
                cond_field = adventureGame.actions[action].conditions.condition_fields
                numbers = re.findall(r'\d+', proc)  # Find all sequences of digits
                max_field_num = str(max(map(int, numbers)) + 1)
                new_field_index = " and FIELD_" + max_field_num
                newProc = proc + new_field_index
                newField = ConditionFieldFactory.create_condition_field(new_attribute)
                cond_field[new_field_index] = newField
                newConditions = ComplexCondition(newProc, cond_field)
                adventureGame.actions[action].conditions = newConditions
        
            
    adventureGame.add_action_template(action_template)
    command = replace_placeholders(base_form, arguments)
    action = adventureGame.get_action(command)
    for flag in preceding_events:
        action.flags.append(flag)
        addNewAction(adventureGame, llm, flag + "; preceding: false.", currRoom)
    assert action is not None, "Bug! Action is none!"
    fixes = action.conditions.get_fixes_complex(adventureGame, llm=llm)
    for fix in fixes:
        fix.apply(adventureGame.world)

    action_result = action.execute(adventureGame)
    adventureGame.happened_events.add(user_input)


    # STEP 6: Check if the action succeeds.

    action_result = adventureGame.action_history[-1]
    if not action_result.success:
        # REMOVE THE ACTION IF ALREADY ADDED
        print("REMOVE NOT WORKING ACTION", adventureGame.remove_action(action_template))
    
    effects = effects.split(';')

    # STEP 7: Check if the action has the correct effects.
    #desired_effects = [f'{{{effect}}}' for effect in effects]
    #actual_effects_expression = ' and '.join(desired_effects)
    #print(actual_effects_expression)
    #actual_effects_condition = ComplexCondition.build_from_string(
    #    game = adventureGame, expression = actual_effects_expression)
    #is_satisfied, messages = actual_effects_condition.evaluate(
    #    adventureGame)
    #print("ACTUAL EFFECTS SATISFACTION CHECK")
    #print(is_satisfied, messages)
    adventureGame.commands.append(command)
    adventureGame.world.restore_initial_state()
    adventureGame.happened_events = set()
    adventureGame.happened_actions = set()
    game_builder.populate_node_attributes()

In [ ]:
# Sample agent to play through games ideally
from type import GameState

class GameAgent:
    def __init__(self, game, game_logics):
        """Initializes agent with game instance and game logistics

        Args:
            game_logics (dict): The game logics containing ordered actions
        """

        self.game = game
        self.game_logics = game_logics['game_logics']
        self.actions = self.game_logics.keys()

    def play(self):
        '''
        Plays through the game by executing each action in game_logics sequentially
        '''
        
        for i in self.actions:
            action = i.replace("adventurer ", "")
            action = action + " " + self.game_logics[i]["location"][0]
            action = re.sub(r'[^\w\s]','', action)
            print("ACTION", action)
            self.game.execute_command(action)
            
            if self.game.game_state != GameState.UNFINISHED:
                print("Game Over")
                if self.game.game_state == GameState.WON:
                    print("You Win")
                else:
                    print("You lose")
                break

        print("\nGame Agent has finished executing all actions.")

# Restore to original state before playing
# adventureGame.world.restore_initial_state()
# adventureGame.world.move_node(adventureGame.world.player, adventureGame.world.find_node("forest"))

myAgent = GameAgent(adventureGame, game_input)
myAgent.play()


# Restore to original state after playing
adventureGame.world.restore_initial_state()
print(myAgent.actions)

In [ ]:
from planner.planner import gptPlanner

title = "Basketball Dreams"
main_quest_line = [
    "Train everyday to get better and better and basketball.",
    "Become the best basketball player in the world."
]
description = "A kid trains to go to the NBA, but doesn't have the equipment at home. He trains everyday, and gets better and better as day goes on."

plannergpt = gptPlanner(llm)
plannergpt.pipeline(title, main_quest_line, description)